In [56]:
import pandas as pd
import csv
import sklearn.linear_model as lm
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [45]:
'''
Utility functions
'''
def split(X, split_size):
    np.random.shuffle(X)
    break_pt = split_size * np.shape(X)[0]
    return X[:break_pt,:], X[break_pt:,:]

# implementation notes: set NaN to mean
def normalize_features(X_train):
    mean_X_train = np.nanmean(X_train, 0)
    for i in xrange(np.shape(X_train)[1]):
        col = X_train[:,i]
        col[ np.isnan(col) ] = mean_X_train[i]
    std_X_train = np.std(X_train, 0)
    std_X_train[ std_X_train == 0 ] = 1
    X_train_normalized = (X_train - mean_X_train) / std_X_train
    return X_train_normalized

# Note: bucket edits in place
def bucket(X, cols, num_buckets):
    Y = np.copy(X)
    for col in cols:
        buckets = np.linspace(np.min(X[:,col]), np.max(X[:,col]), num=num_buckets+1)
        for i in xrange(num_buckets):
            X_col = Y[:,col]
            X_col[ (buckets[i] <= X_col) & (X_col <= buckets[i+1])] = i
            Y[:,col] = X_col
    return Y

def rmse(predict, true):
    return np.sqrt(1.0/np.shape(predict)[0] * np.sum(np.square(predict - true)))

In [50]:
'''
Load in data
'''
import re
import warnings

years = 2000 + np.linspace(1,15, 15)
buckets = np.zeros(1)
target_type = str  # The desired output type

for year in years:

    data_file = '../data/Crimes_-_%d.csv' % year

    with warnings.catch_warnings(record=True) as ws:
        warnings.simplefilter("always")

        data = pd.read_csv(data_file, sep=",", header=0)
        print("Warnings raised:", ws)
        # We have an error on specific columns, try and load them as string
        for w in ws:
            s = str(w.message)
            print("Warning message:", s)
            match = re.search(r"Columns \(([0-9,]+)\) have mixed types\.", s)
            if match:
                columns = match.group(1).split(',') # Get columns as a list
                columns = [int(c) for c in columns]
                print("Applying %s dtype to columns:" % target_type, columns)
                data.iloc[:,columns] = data.iloc[:,columns].astype(target_type)

    '''
    Extract Relevant Features
    '''
    # temporal features
    date_time = np.array([x.split() for x in data.Date])
    date = date_time[:,0]
    time = date_time[:,1]
    tod = date_time[:,2]

    # month, day, year
    date = np.array([x.split('/') for x in date])
    month = [int(x) for x in date[:,0]]
    dom = [int(x) for x in date[:,1]]
    years = [int(x) for x in date[:,2]]
    time_feat = np.subtract(years, 2001)*12 + month

    # time of day
    time_c = [x.split(':') for x in time]
    time = [int(x[1]) if (y == 'AM' and int(x[0]) == 12) else 60*int(x[0])+int(x[1]) 
            if (y =='AM' and int(x[0]) != 12) or (int(x[0]) == 12 and y == 'PM') else 12*60+60*int(x[0])+int(x[1]) 
            for x,y in zip(time_c, tod)]

    feats = np.transpose(np.vstack((time_feat, data.Latitude, data.Longitude))).astype(float)

    # remove NaNs
    feats = feats[~(np.isnan(feats[:,1]))]

    # bucket the data
    n_buckets = 5
    data_b = bucket(feats, [1, 2], n_buckets)
    
    n_time = 12 #int(data_b[np.argmax(data_b[:,0])][0])

    #buckets = np.zeros((n_time, n_buckets, n_buckets))
    buckets2 = np.zeros((n_buckets * n_buckets * n_time, 4))

    # count the data per geographic cell per month
    for i in xrange(n_time):
        for j in xrange(n_buckets):
            for k in xrange(n_buckets):

                # note: one-indexing for months
                mo = i+1+(year-2001)*12
                count = data_b[ (data_b[:,0] == mo) & 
                                (data_b[:,1] == j) & 
                                (data_b[:,2] == k) ]
    #            buckets[i][j][k] = np.size(count,0)
                buckets2[i*(n_buckets * n_buckets)+j*(n_buckets)+k][0] = mo
                buckets2[i*(n_buckets * n_buckets)+j*(n_buckets)+k][1] = j
                buckets2[i*(n_buckets * n_buckets)+j*(n_buckets)+k][2] = k
                buckets2[i*(n_buckets * n_buckets)+j*(n_buckets)+k][3] = np.size(count,0)
    
    if np.size(buckets) == 1:
        buckets = buckets2
    else:
        buckets = np.vstack((buckets, buckets2))
    
    print "finished", year

('Warnings raised:', [])
finished 2001.0
('Warnings raised:', [])
finished 2002.0
('Warnings raised:', [])
finished 2003.0
('Warnings raised:', [])
finished 2004.0
('Warnings raised:', [])
finished 2005.0
('Warnings raised:', [])
finished 2006.0
('Warnings raised:', [])
finished 2007.0
('Warnings raised:', [])
finished 2008.0
('Warnings raised:', [])
finished 2009.0
('Warnings raised:', [])
finished 2010.0
('Warnings raised:', [])
finished 2011.0
('Warnings raised:', [])
finished 2012.0
('Warnings raised:', [])
finished 2013.0
('Warnings raised:', [])
finished 2014.0
('Warnings raised:', [])
finished 2015.0


In [58]:
with open("chicago_feats.pkl", "wb") as f:
    pickle.dump(buckets, f)